In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")) + '/lib/')
import cv2
import time
import torch
import numpy as np
import pandas as pd
from torch_geometric.data import Data
from lib.blob_extraction import img_preprocess, blob_detect, get_nodes_pos
from lib.graph_generate import Delaunay_graph_generate
from lib.voronoi_generate import TransformVoronoi_331

## 1. Define Voronoi Graph Dataset

In [2]:
def voronoi_graph_datatset_generate(raw_data_dir, train_data_dir, test_data_dir, tip_num = 331):
    
    ####  1. Presetting  ####

    os.makedirs(train_data_dir, exist_ok=True)  ## make dir for saving train/val data
    os.makedirs(test_data_dir, exist_ok=True)  ## make dir for saving test data 

    raw_data_label = pd.read_csv(os.path.join(os.path.abspath(os.path.join(raw_data_dir, "..")), 'targets.csv'), header=0, usecols=['image_name','pose_2', 'pose_6'])  ## read label.csv file
    data_label = np.array([raw_data_label['image_name'].values,
                       raw_data_label['pose_2'].values,
                       raw_data_label['pose_6'].values])

    data_label = data_label.T  ## images index with labels (pose2--Y & pose6--Theta_roll)
    count = 1  ## record total generated graph number
    data_list = []  ## list for saving the graph data
    pin_extract_fail = []  ## list for saving the image index which fails of pin extraction 
    voronoi_fail = []  ## list for saving the image index which fails of voronoi generation 
    start = time.time()


    ####  2. Generating  ####

    for i in range(len(data_label)):

        image_name = data_label[i][0]

        image_dir = raw_data_dir + '/' + image_name

        img = cv2.imread(image_dir)  ## get raw image
    
        ## !!! tune the parameters by yourself, refer to the setup process in the last step !!!
        processed = img_preprocess(img, 
                                   erosion = False,
                                   kernel_size = 1,
                                   resize_x = 300, 
                                   resize_y = 300,
                                   binary_threshold = 100, 
                                   circle_x_bias = -1, 
                                   circle_y_bias = -2, 
                                   circle_radius_bias = -14)
        
        ## !!! tune the parameters by yourself, refer to the setup process in the last step !!!
        keypoints = blob_detect(processed, minArea = 10, blobColor = 255, 
                                minCircularity = 0.01, minConvexity = 0.01, 
                                minInertiaRatio = 0.01, thresholdStep = 5,
                                minDistBetweenBlobs = 3.0, minRepeatability = 3)
 
        nodes_pos = get_nodes_pos(keypoints)  ## extracted nodes position

        if nodes_pos.shape[0] == tip_num:  ## check nodes size

            Axx_canon, Cxx_canon, Cyy_canon, XY_canon = TransformVoronoi_331(borderScale=1.1).transform(nodes_pos)  ## voronoi construction

            if len(Axx_canon) == tip_num:  ## check voronoi area size

                label = [data_label[i][1],
                        data_label[i][2]]  ## pose2--Y & pose6--Theta_roll

                nodes_edges = Delaunay_graph_generate(nodes_pos)  ## Delaunay edge construction

                edge = torch.tensor(np.array(nodes_edges).T, dtype=torch.long)  ## convert graph edge to tensor

                pair = np.asarray([XY_canon[:,0], XY_canon[:,1], Axx_canon[:]]).T  ## graph feature pair, (nodes x, nodes y, voronoi cell area)

                x = torch.tensor(pair, dtype=torch.float)  ## convert graph node feature to tensor

                d = Data(x=x, edge_index=edge.contiguous(), t=torch.tensor(label))  ## load nodes, edges and labels into Data format

                data_list.append(d) ## update graph data list

                count = count + 1  ## update total generated graph number

                print("\rRaw Image Data loaded " + str(i+1), end="  ")

            else:

                voronoi_fail.append(i)  ## update list for saving the image index which fails of pin extraction 

                print("\rRaw Image Data loaded " + str(i+1), end="  ")

                continue

        else:

            pin_extract_fail.append(i)  ## update list for saving the image index which fails of voronoi generation 

            print("\rRaw Image Data loaded " + str(i+1), end="  ")

            continue

    
    ####  3. Saving  ####

    np.random.shuffle(data_list)  ## shuffle the graph data list
    data_size = len(data_list)
    train_val_size = int(data_size*0.75)  ## size of train/val dataset, 75%
    test_size = int(data_size*0.25)  ## size of test dataset, 25%

    train_val_data = data_list[:train_val_size]  ## train/val dataset
    test_data = data_list[train_val_size+1:]  ## test dataset

    train_val_data_file = train_data_dir + '/Train_val_data_list.pt'
    torch.save(train_val_data, train_val_data_file)  # save the train_val data_list as pt file

    test_data_file = test_data_dir + '/Test_data_list.pt'
    torch.save(test_data, test_data_file)  # save the test data_list as pt file

    cost_time = time.time() - start
    print("\nVoronoi Graph Dataset Loading Complete!!!")
    print("The shape of processed images = ", processed.shape)
    print("The whole number of processed images = ", count)
    print("The whole number of train/validation dataset = ", train_val_size)
    print("The whole number of test dataset = ", test_size)
    print("The fail number of pin extraction = ", len(pin_extract_fail))
    print("The fail number of voronoi generation = ", len(voronoi_fail))
    print("The whole time of voronoi graph construction = ", cost_time)
    print("The voronoi graph construction frequency = ", count/cost_time)
    print("Each voronoi graph construction time = ", cost_time/count)
    print("Image preprocessing and Voronoi Graph generating successfully!")

    return train_val_data, test_data

## 2. Generate Voronoi Graph Dataset

In [3]:
raw_data_dir = r'..\data\331\model_surface2d\frames_bw'  ## raw image dir
train_data_dir = r'..\result\train'  ## train data saving dir
test_data_dir = r'..\result\test'  ## test data saving dir

## run the dataset generation...
voronoi_graph_data_list = voronoi_graph_datatset_generate(raw_data_dir, train_data_dir, test_data_dir, tip_num = 331)

Raw Image Data loaded 5000  
Voronoi Graph Dataset Loading Complete!!!
The shape of processed images =  (300, 300)
The whole number of processed images =  4841
The whole number of train/validation dataset =  3630
The whole number of test dataset =  1210
The fail number of pin extraction =  160
The fail number of voronoi generation =  0
The whole time of voronoi graph construction =  258.6172387599945
The voronoi graph construction frequency =  18.71878310669232
Each voronoi graph construction time =  0.05342227613302923
Image preprocessing and Voronoi Graph generating successfully!
